## Start

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
from imports import *
from fast_gen import preprocess_scale, scale_and_center

Using TensorFlow backend.


In [13]:
imagenet_mean = np.array([103.939, 116.779, 123.68], dtype=np.float32).reshape((1,1,3))
def preprocess_imagenet(x): return x[..., ::-1] - imagenet_mean

In [3]:
bs=64; sz=224; lr=2e-3

In [4]:
path = "/data/jhoward/fast/dogscats/"

In [5]:
mn=VGG16()
conv_outp=mn.get_layer('predictions').input
outp = Dense(2, activation='softmax')(conv_outp)
m = Model(mn.input, outp)
for l in m.layers[:-1]: l.trainable=False
m.compile(SGD(lr, momentum=0.9), 'categorical_crossentropy', metrics=['accuracy'])

In [9]:
gen = image.ImageDataGenerator(preprocessing_function=preprocess_imagenet)
trn_batches = gen.flow_from_directory(f'{path}train', (sz,sz), batch_size=bs)
fix_batches = gen.flow_from_directory(f'{path}train', (sz,sz), batch_size=bs, shuffle=False)
val_batches = gen.flow_from_directory(f'{path}valid', (sz,sz), batch_size=bs, shuffle=False)
nb_trn = math.ceil(trn_batches.n/bs)
nb_val = math.ceil(val_batches.n/bs)

Found 23000 images belonging to 2 classes.
Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
m.fit_generator(trn_batches, nb_trn, workers=1, epochs=3,
                 validation_data=val_batches, validation_steps=nb_val)

Epoch 1/3
360/360 [==============================] - 136s - loss: 0.0698 - acc: 0.9772 - val_loss: 0.0515 - val_acc: 0.9805
Epoch 2/3
360/360 [==============================] - 132s - loss: 0.0400 - acc: 0.9859 - val_loss: 0.0441 - val_acc: 0.9830
Epoch 3/3
360/360 [==============================] - 132s - loss: 0.0294 - acc: 0.9891 - val_loss: 0.0425 - val_acc: 0.9850


In [17]:
mn=VGG16()
conv_outp=mn.get_layer('predictions').input
m = Model(mn.input, conv_outp)

In [18]:
fix_batches.reset(); val_batches.reset()
trn_acts = m.predict_generator(generator=fix_batches, verbose=1, 
                                steps=nb_trn, workers=1)
val_acts = m.predict_generator(generator=val_batches, verbose=1,
                                steps=nb_val, workers=1)

32/32 [==============================] - 10s    


In [20]:
inp = Input(batch_shape=m.output_shape)
outp = Dense(1, activation='sigmoid')(inp)
fc = Model(inp, outp)
fc.compile(SGD(lr, momentum=0.9), 'binary_crossentropy', metrics=['accuracy'])

In [21]:
fc.fit(trn_acts, fix_batches.classes, bs, 3, validation_data=(val_acts, val_batches.classes))

Train on 23000 samples, validate on 2000 samples
Epoch 1/3
23000/23000 [==============================] - 1s - loss: 0.0605 - acc: 0.9773 - val_loss: 0.0556 - val_acc: 0.9820
Epoch 2/3
23000/23000 [==============================] - 1s - loss: 0.0375 - acc: 0.9857 - val_loss: 0.0378 - val_acc: 0.9845
Epoch 3/3
23000/23000 [==============================] - 1s - loss: 0.0306 - acc: 0.9888 - val_loss: 0.0366 - val_acc: 0.9845
